# Debugging

## Postmortem debugging

I wrote this great code! Unfortunately it is going to crash (there will be an unhandled exception)...

But afterwards I can inspect the stack at the time using `pdb.pm`.

When there is an unhandled exception, Python stores a bunch of information in [sys.last_traceback](https://docs.python.org/3.2/library/sys.html#sys.last_traceback). This is where `pdb.pm` pulls the info from.

In [1]:
a = 2
b = [1,2,3]
a + b

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [2]:
import pdb; pdb.pm()

> <ipython-input-1-0eae50b1b6cc>(3)<module>()
-> a + b
(Pdb) print(a)
2
(Pdb) print(b)
[1, 2, 3]
(Pdb) print("Ahh")
Ahh
(Pdb) exit


## Auto postmortem debugging

If you always want to get dropped into a debugger on an error you can set that up with the pdb magic. I find this a little bit annoying (often the error is obvious and I don't need the debugger and typing `exit` is so hard...) so tend to leave it off.

In [14]:
%pdb 1
a + b
%pdb 0

Automatic pdb calling has been turned ON


TypeError: unsupported operand type(s) for +: 'int' and 'list'

None
> <ipython-input-14-89bf6873407c>(2)<module>()
      1 get_ipython().run_line_magic('pdb', '1')
----> 2 a + b
      3 get_ipython().run_line_magic('pdb', '0')

ipdb> exit


## Premortem 

Sometimes you are writing code that you just want to check really thoroughly to make sure everything is happening as you imagined. Premortem debugging (stepping though) is useful then.

Slightly weird things happen if you put the `pdb.set_trace` outside of a function. The `n` steps into `IPython.core` code which you probably don't want. See [here](https://stackoverflow.com/questions/46495269/debuggers-not-acting-properly-on-jupyter-notebooks).

If you do want to start traces outside of functions (a totally reasonable thing!) I would suggest just setting a breakpoint on the next line.

In [1]:
def binarize(x, threshold):
    if x > threshold:
        return 1
    else:
        return 0

# Ahh this is to complicated for me to think all the way through
import pdb;pdb.set_trace()
binarize(2, 1)


--Return--
> <ipython-input-1-a87650b633d8>(8)<module>()->None
-> import pdb;pdb.set_trace()
(Pdb) l
  3  	        return 1
  4  	    else:
  5  	        return 0
  6  	
  7  	# Ahh this is to complicated for me to think all the way through
  8  ->	import pdb;pdb.set_trace()
  9  	binarize(2, 1)
[EOF]
(Pdb) b 9
Breakpoint 1 at <ipython-input-1-a87650b633d8>:9
(Pdb) c
> <ipython-input-1-a87650b633d8>(9)<module>()->None
-> binarize(2, 1)
(Pdb) s
--Call--
> <ipython-input-1-a87650b633d8>(1)binarize()
-> def binarize(x, threshold):
(Pdb) print("etc etc")
etc etc
(Pdb) exit


BdbQuit: 